# Unfiltered cells / rewrite of COS + fig generation

In [ ]:
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import pingouin as pg
from scipy.stats import spearmanr
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import quilt3

from fish_morphology_code.analysis.notebook_utils import (
    DAY_18_COLOR,
    DAY_32_COLOR,
    DAY_COLOR_PALETTE,
    DAY_COLOR_PALETTE_THREE,
    BAR_PLOT_COLUMNS,
    SHORT_FEAT_NAME_MAP,
    BAR_PLOT_COLUMNS_SHORT,
    PROBE_ORDER,
    CI_EXTENT,
    FEATURE_TYPE_MAP,
    safe,
    get_regression_coef,
    boot_regression,
    ci_low,
    ci_high,
    make_reg_plot_ci_df,
    make_regression_bar_plot,
    get_pred_true,
    make_regression_scatter_plot,
    boot_spearmanr,
)

In [ ]:
SAVE=True

if SAVE:
    SAVE_DIR = Path("./plots_nofilter")
    SAVE_DIR.mkdir(parents=True, exist_ok=True)

    save_dir_pngs = SAVE_DIR/"pngs"
    save_dir_svgs = SAVE_DIR/"svgs"
    save_dir_pngs.mkdir(parents=True, exist_ok=True)
    save_dir_svgs.mkdir(parents=True, exist_ok=True)

## Load data

### New fish data from tanya

In [ ]:
df_plates = pd.read_csv("../paper_fish_plates_20201027.csv", skipfooter=1)
df_plates["Plate Name"] = df_plates["Plate Name"].astype(int)
df_plates = df_plates.rename(columns={"Plate Name":"plate_name", "Replate Date":"replate_date"})
df_plates

In [ ]:
df_fish_old_new_full = pd.read_csv(
    "/Users/rorydm/projects/fish_morphology_manuscript_notebooks/fish_all_features_20201013.csv",
    low_memory=False,
)

df_fish_old_new_full
df_fish_old_new_full.napariCell_ObjectNumber = df_fish_old_new_full.napariCell_ObjectNumber.astype(int).astype(str)
df_fish_old_new_full["Type"] = "FISH"

### ACTN2 fish data from quilt

In [ ]:
p_fish_actn2 = quilt3.Package.browse(
    "tanyasg/2d_autocontrasted_single_cell_features_actn2",
    registry="s3://allencell-internal-quilt"
)

df_fish_actn2_full = p_fish_actn2["features/5a192f08_cp_features.csv"]()
df_fish_actn2_full["Dataset"] = "ACTN_OldFish"

In [ ]:
p_fish_actn2_2 = quilt3.Package.browse(
    "tanyasg/2d_autocontrasted_single_cell_features_actn2_2",
    registry="s3://allencell-internal-quilt"
)

df_fish_actn2_2_full = p_fish_actn2_2["features/e1c7dd15_cp_features.csv"]()
df_fish_actn2_2_full["Dataset"] = "ACTN_NewFish"

In [ ]:
df_fish_actn2_both_full = pd.concat([df_fish_actn2_full, df_fish_actn2_2_full]).reset_index(drop=True)
df_fish_actn2_both_full.napariCell_ObjectNumber = df_fish_actn2_both_full.napariCell_ObjectNumber.astype(int).astype(str)

df_fish_actn2_both_full = df_fish_actn2_both_full.merge(df_plates[["plate_name", "replate_date"]])
df_fish_actn2_both_full["Type"] = "FISH"
df_fish_actn2_both_full["ge_wellID"] = df_fish_actn2_both_full.plate_name.astype(str) + "-" + df_fish_actn2_both_full.well_position.astype(str)

### ACTN2 struct scores

In [ ]:
p_struct_scores_actn2 = quilt3.Package.browse(
    "tanyasg/struct_scores_actn2",
    registry="s3://allencell-internal-quilt"
)

df_struct_scores_actn2 = p_struct_scores_actn2["metadata.csv"]()
df_struct_scores_actn2 = df_struct_scores_actn2.rename(columns={"original_fov_location":"fov_path"})
df_struct_scores_actn2.napariCell_ObjectNumber = df_struct_scores_actn2.napariCell_ObjectNumber.astype(int).astype(str)
df_struct_scores_actn2 = df_struct_scores_actn2.drop(columns=["Age"])

### merge fish with live classifier stuff

In [ ]:
df_actn2 = df_fish_actn2_both_full.merge(df_struct_scores_actn2)
df_actn2.shape

## combine old/new fish + classifier with actn2 fish + classifier

In [ ]:
df_fish_all = pd.concat(
    [
        df_fish_old_new_full,
        df_actn2
    ]
).reset_index(drop=True)

feature_columns_in = [
    "napariCell_AreaShape_Area",
    "napariCell_AreaShape_MinorAxisLength",
    "napariCell_AreaShape_MajorAxisLength",
    'Frac_Area_Background',
    'Frac_Area_DiffuseOthers',
    'Frac_Area_Fibers',
    'Frac_Area_Disorganized_Puncta',
    'Frac_Area_Organized_Puncta',
    'Frac_Area_Organized_ZDisks',
    'Maximum_Coefficient_Variation',
    'Peak_Height',
    'Peak_Distance',
]
metadata_cols_in = [
    "Dataset",
    "napariCell_ObjectNumber",
    "fov_path",
    "cell_age",
    'replate_date',
    "image_date",
    "plate_name",
    "well_position",
    "ge_wellID",
    "mh_structure_org_score",
    "kg_structure_org_score",
    "MH_score",
    "KG_score",
    "Type",
    'probe546',
    'probe647',
    "napariCell_Children_seg_probe_561_Count",
    "napariCell_Children_seg_probe_638_Count",
    'IntensitySumIntegrated',
    'IntensitySumIntegratedBkgSub',
]

df_fish_all = df_fish_all[metadata_cols_in+feature_columns_in]

for i, row in df_fish_all.dropna(subset=["MH_score","KG_score"]).iterrows():
    df_fish_all.at[i,"mh_structure_org_score"] = row["MH_score"]
    df_fish_all.at[i,"kg_structure_org_score"] = row["KG_score"]
df_fish_all = df_fish_all.drop(columns=["MH_score","KG_score"])

df_fish_all["Cell aspect ratio"] = (
    df_fish_all["napariCell_AreaShape_MinorAxisLength"]
    / df_fish_all["napariCell_AreaShape_MajorAxisLength"]
)
df_fish_all = df_fish_all.drop(
    columns=["napariCell_AreaShape_MinorAxisLength", "napariCell_AreaShape_MajorAxisLength"]
)

metadata_cols_out = [c for c in metadata_cols_in if c not in ["MH_score", "KG_score"]]
feature_columns_out = [c for c in df_fish_all.columns if c not in metadata_cols_out]

In [ ]:
for col in ["probe546", "probe647"]:
    df_fish_all[col] = df_fish_all[col].apply(lambda x: x.split("-")[0])

In [ ]:
df_fish_all = df_fish_all.rename(
    columns={
        'IntensitySumIntegrated': "alpha-actinin protein intensity (sum)",
        'IntensitySumIntegratedBkgSub': "alpha-actinin protein intensity (sum, background subtracted)",
    }
)

In [ ]:
n_valid_fishes_in = (~df_fish_all[["probe546","probe647"]].isnull()).sum().sum()
n_valid_fishes_in

In [ ]:
def unmelt_probes(arg_df):
    
    out_df = arg_df.copy()
    
    for probe in np.unique(out_df[["probe546", "probe647"]].dropna().values):
        out_df[f"{probe}_count"] = np.nan
    
    for i, row in out_df.iterrows():
        probe546 = row["probe546"]
        probe647 = row["probe647"]
        out_df.at[i, f"{probe546}_count"] = row["napariCell_Children_seg_probe_561_Count"]
        out_df.at[i, f"{probe647}_count"] = row["napariCell_Children_seg_probe_638_Count"]

    out_df = out_df.drop(
        [
            "probe546",
            "probe647",
            "napariCell_Children_seg_probe_561_Count",
            "napariCell_Children_seg_probe_638_Count",
        ],
        axis="columns"
    )
    
    return out_df

## give all probe counts their own columns

In [ ]:
df_fish_all = unmelt_probes(df_fish_all)
df_fish_all.shape

In [ ]:
n_valid_fishes_out = (~df_fish_all[[c for c in df_fish_all.columns if c.endswith("_count")]].isnull()).sum().sum()
n_valid_fishes_out

In [ ]:
assert n_valid_fishes_out == n_valid_fishes_in

## move to density rather than counts

In [ ]:
count_cols = [c for c in df_fish_all.columns if c.endswith("count")]

for count_col in count_cols:
    probe = count_col.split("_")[0]
    density_col = f"{probe}_density"
    df_fish_all[density_col] = df_fish_all[count_col]/df_fish_all.napariCell_AreaShape_Area
    
df_fish_all = df_fish_all.drop(columns=count_cols)

In [ ]:
df_fish_all.isnull().mean()

## Live cell data from tanya

In [ ]:
df_live = pd.read_csv("../classifier_features_metadata_20201028.csv")

live2fish_rename_dict = {
    'diffuse_fraction_area_covered': 'Frac_Area_DiffuseOthers',
    'fibers_fraction_area_covered': 'Frac_Area_Fibers',
    'disorganized_puncta_fraction_area_covered': 'Frac_Area_Disorganized_Puncta',
    'organized_puncta_fraction_area_covered': 'Frac_Area_Organized_Puncta',
    'organized_z_disk_fraction_area_covered': 'Frac_Area_Organized_ZDisks',
    'Aspect_Ratio': 'Cell aspect ratio',
    'Area':'napariCell_AreaShape_Area',
    'MH_score':'mh_structure_org_score',
    'KG_score':'kg_structure_org_score',
    'Cell age': 'cell_age',
    'FOV path': 'fov_path',
    'cell_id':'napariCell_ObjectNumber',
    'full_cell_total_sum_intensity':"alpha-actinin protein intensity (sum)",
    'full_cell_total_sum_intensity_bg_subtracted':"alpha-actinin protein intensity (sum, background subtracted)",
}

df_live = df_live.rename(
    columns=live2fish_rename_dict
)

# fill nans in feature cols with zeros????
df_live[feature_columns_out] = df_live[feature_columns_out].fillna(0)

df_live["ge_wellID"] = df_live.plate_name.astype(str) + "-" + df_live.well_position.astype(str)
df_live["Dataset"] = "Live"
df_live["Type"] = "Live"

### Combine FISH + Live

In [ ]:
df = pd.concat(
    [
        df_fish_all,
        df_live[[c for c in df_live.columns if c in df_fish_all.columns]],
    ]
).reset_index(drop=True)

# fix units on length / area cols
pixel_size_xy_in_micrometers=0.12
df["napariCell_AreaShape_Area"] = df["napariCell_AreaShape_Area"] * pixel_size_xy_in_micrometers**2
df["Peak_Distance"] = df["Peak_Distance"] * pixel_size_xy_in_micrometers

density_cols = [c for c in df.columns if c.endswith("_density")]
for col in density_cols:
    df[col] = df[col] / pixel_size_xy_in_micrometers**2

In [ ]:
df = df.rename(
    columns={
        "napariCell_AreaShape_Area": 'Cell area (μm^2)',
        "Frac_Area_Background": 'Fraction cell area background',
        "Frac_Area_DiffuseOthers": 'Fraction cell area diffuse/other',
        "Frac_Area_Fibers": 'Fraction cell area fibers',
        "Frac_Area_Disorganized_Puncta": 'Fraction cell area disorganized puncta',
        "Frac_Area_Organized_Puncta": 'Fraction cell area organized puncta',
        "Frac_Area_Organized_ZDisks": 'Fraction cell area organized z-disks',
        "Maximum_Coefficient_Variation": 'Max coefficient var',
        "Peak_Height": 'Peak height',
        "Peak_Distance": 'Peak distance (μm)',
        "cell_age": "Cell age",
        **{col: f"{col.split('_')[0]} (count/μm^2)"for col in density_cols}
    }
)

In [ ]:
df['Expert structural annotation score (mean)'] = df[
    [
        "mh_structure_org_score",
        "kg_structure_org_score"
    ]
].mean(
    axis="columns"
)

In [ ]:
df["Cell age"] = df["Cell age"].map(
    {
        18:18,
        19:18, 
        25:25,
        26:25,
        32:32,
        33:32,
    }
)

In [ ]:
df.isnull().mean()

#### oldfish is in here twice as "Transcript-Protein"

In [ ]:
assert all(df[df.Dataset == "Transcript-Protein"][
    ["napariCell_ObjectNumber", "fov_path"]
].drop_duplicates().reset_index(drop=True) == df[df.Dataset == "ACTN_OldFish"][
    ["napariCell_ObjectNumber", "fov_path"]
].drop_duplicates().reset_index(drop=True))

In [ ]:
df = df.drop(df[df.Dataset == "Transcript-Protein"].index).reset_index(drop=True)

In [ ]:
assert len(df) == len(df[["napariCell_ObjectNumber", "fov_path"]].drop_duplicates().reset_index(drop=True))

## Fit regression on old FISH data to predict on all others

In [ ]:
df_old = df[df.Dataset=="OldFish"].copy()

all_good_scores = (df_old.kg_structure_org_score > 0) & (df_old.mh_structure_org_score > 0)
df_old = df_old[all_good_scores]
df_old.shape

In [ ]:
from sklearn.preprocessing import StandardScaler

feat_cols = [c for c in BAR_PLOT_COLUMNS if c in df.columns]

df_reg = df_old[feat_cols+['Expert structural annotation score (mean)']].copy()

scaler = StandardScaler()
scaler.fit(df_reg[feat_cols])

df_reg[feat_cols] = scaler.transform(df_reg[feat_cols])

### Define regression procedure

In [ ]:
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor

def make_regression(
    scaled_data=pd.DataFrame(),
    y_col='Expert structural annotation score (mean)',
    X_cols=feat_cols,
    weight_y=True,
    alpha=0.001,
):
    X = scaled_data[X_cols]        
    y = scaled_data[y_col]

    if weight_y:
        class_weights = {
            v: len(y) / c for v, c in zip(*np.unique(y, return_counts=True))
        }
        sample_weights = scaled_data[y_col].map(class_weights)
    else:
        sample_weights = 1

    reg = linear_model.Ridge(alpha=alpha)
    reg.fit(X, y, sample_weight=sample_weights)
    
    return reg


def make_regression_rf(
    scaled_data=pd.DataFrame(),
    y_col='Expert structural annotation score (mean)',
    X_cols=feat_cols,
    weight_y=True,
    **rf_kwargs,
):
    X = scaled_data[X_cols]        
    y = scaled_data[y_col]

    if weight_y:
        class_weights = {
            v: len(y) / c for v, c in zip(*np.unique(y, return_counts=True))
        }
        sample_weights = scaled_data[y_col].map(class_weights)
    else:
        sample_weights = 1

    reg = RandomForestRegressor(**rf_kwargs)
    reg.fit(X, y, sample_weight=sample_weights)
    
    return reg

### Predict COS for all cells

In [ ]:
reg = make_regression(df_reg)
df["Combined organizational score"] = reg.predict(scaler.transform(df[feat_cols]))

### Check predictions

In [ ]:
df_expert = df[
    (df.kg_structure_org_score > 0) & (df.mh_structure_org_score > 0)
][
    [
        "kg_structure_org_score",
        "mh_structure_org_score",
        'Expert structural annotation score (mean)',
        'Combined organizational score',
        "Dataset",
        'Cell age',
    ]
]
df_expert['Combined organizational score (rounded)'] = np.round(np.clip(df_expert['Combined organizational score'], 1,5))

df_expert_melt = df_expert.melt(
    value_vars=["kg_structure_org_score", "mh_structure_org_score"],
    id_vars=[c for c in df_expert.columns if c not in ["kg_structure_org_score", "mh_structure_org_score"]],
    var_name="Expert annotator",
    value_name="Expert score"
)
df_expert_melt["Expert annotator"] = df_expert_melt["Expert annotator"].apply(lambda x: x.split("_")[0])
df_expert_melt["Expert score"].unique()

assert np.all(sorted(df_expert_melt["Expert score"].unique()) == np.array([1,2,3,4,5]))
assert np.all(sorted(df_expert["kg_structure_org_score"].unique()) == np.array([1,2,3,4,5]))
assert np.all(sorted(df_expert["mh_structure_org_score"].unique()) == np.array([1,2,3,4,5]))

In [ ]:
g = sns.FacetGrid(
    data=df_expert_melt.sample(frac=1, replace=False).reset_index(drop=True),
    col="Dataset",
    hue="Cell age",
    hue_order=[18, 32],
    palette=DAY_COLOR_PALETTE,
)

g = g.map(
    sns.stripplot,
    "Expert score",
    "Combined organizational score",
    order=[1,2,3,4,5],
    alpha=0.2,
    linewidth=0,
    jitter=0.25,
    s=3
).add_legend()

In [ ]:
df_corrs_exp_both_cos = df_expert_melt.groupby(
    "Dataset"
)[
    [
        "Expert score",
        'Combined organizational score'
    ]
].corr(method="spearman").drop(
    columns=['Expert score']
).rename_axis(
    ['Dataset', 'column']
).drop(
    "Combined organizational score",
    level='column'
).reset_index(
).drop(
    columns="column"
).rename(columns={"Combined organizational score":"Expert-COS correlation (both)"})

df_corrs_exp_mean_cos = df_expert.groupby(
    "Dataset"
)[
    [
        "Expert structural annotation score (mean)",
        'Combined organizational score'
    ]
].corr(method="spearman").drop(
    columns=['Expert structural annotation score (mean)']
).rename_axis(
    ['Dataset', 'column']
).drop(
    "Combined organizational score",
    level='column'
).reset_index(
).drop(
    columns="column"
).rename(columns={"Combined organizational score":"Expert-COS correlation (mean)"})

df_corrs_exp_exp = df_expert.groupby(
    "Dataset"
)[
    [
        "kg_structure_org_score",
        "mh_structure_org_score"
    ]
].corr(method="spearman").drop(
    columns=['kg_structure_org_score']
).rename_axis(
    ['Dataset', 'column']
).drop(
    "mh_structure_org_score",
    level='column'
).reset_index(
).drop(
    columns="column"
).rename(columns={"mh_structure_org_score":"Expert-Expert correlation"})

df_score_corrs = df_corrs_exp_exp.merge(df_corrs_exp_mean_cos).merge(df_corrs_exp_both_cos)
df_score_corrs

### expert-expert confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(df_expert['kg_structure_org_score'], df_expert['mh_structure_org_score'])

dims = (4, 4)
fig, ax = plt.subplots(figsize=dims)
conf_mat_plot = sns.heatmap(
    conf_mat,
    annot=True,
    fmt="d",
    square=True,
    cmap="Blues",
    vmin=0,
    vmax=1000,
    cbar=False,
)

conf_mat_plot.set(
    xlabel='MH score',
    ylabel='KG score',
);
conf_mat_plot.set_xticklabels([1,2,3,4,5])
conf_mat_plot.set_yticklabels([1,2,3,4,5]);

# save png and svg
if SAVE:
    plt.savefig(save_dir_pngs/'expert_expert_conf_mat.png', dpi=300, bbox_inches = "tight")
    plt.savefig(save_dir_svgs/'expert_expert_conf_mat.svg', format="svg", bbox_inches = "tight")

In [ ]:
for dataset in df_expert.Dataset.unique():
    conf_mat = confusion_matrix(
        df_expert[df_expert.Dataset==dataset]['kg_structure_org_score'],
        df_expert[df_expert.Dataset==dataset]['mh_structure_org_score']
    )

    dims = (4, 4)
    fig, ax = plt.subplots(figsize=dims)
    conf_mat_plot = sns.heatmap(
        conf_mat,
        annot=True,
        fmt="d",
        square=True,
        cmap="Blues",
        vmin=0,
        vmax=1000,
        cbar=False,
    )

    conf_mat_plot.set(
        xlabel='MH score',
        ylabel='KG score',
    );
    conf_mat_plot.set_xticklabels([1,2,3,4,5])
    conf_mat_plot.set_yticklabels([1,2,3,4,5]);

    # save png and svg
    if SAVE:
        plt.savefig(save_dir_pngs/f"expert_expert_conf_mat_{dataset}.png", dpi=300, bbox_inches = "tight")
        plt.savefig(save_dir_svgs/f"expert_expert_conf_mat_{dataset}.svg", format="svg", bbox_inches = "tight")

### expert (mean) - COS correlation matrix

In [ ]:
half_int_rows = ~df_expert['Expert structural annotation score (mean)'].isin([1,2,3,4,5])

df_expert['Expert structural annotation score (mean), random round up/down'] = df_expert['Expert structural annotation score (mean)'].copy()
df_expert.loc[
    half_int_rows,'Expert structural annotation score (mean), random round up/down'
] = df_expert.loc[
    half_int_rows,'Expert structural annotation score (mean), random round up/down'
].apply(lambda x: int(x+np.random.rand()))

In [ ]:
from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(
    df_expert['Expert structural annotation score (mean), random round up/down'],
    df_expert['Combined organizational score (rounded)']
)

dims = (4, 4)
fig, ax = plt.subplots(figsize=dims)
conf_mat_plot = sns.heatmap(
    conf_mat,
    annot=True,
    fmt="d",
    square=True,
    cmap="Blues",
    vmin=0,
    vmax=1000,
    cbar=False,
)

conf_mat_plot.set(
    xlabel='Combined organizational score (rounded)',
    ylabel='Expert score (mean)',
);
conf_mat_plot.set_xticklabels([1,2,3,4,5])
conf_mat_plot.set_yticklabels([1,2,3,4,5]);

# save png and svg
if SAVE:
    plt.savefig(save_dir_pngs/'expert_mean_COS_conf_mat.png', dpi=300, bbox_inches = "tight")
    plt.savefig(save_dir_svgs/'expert_mean_COS_conf_mat.svg', format="svg", bbox_inches = "tight")

In [ ]:
for dataset in df_expert.Dataset.unique():

    conf_mat = confusion_matrix(
        df_expert[df_expert.Dataset==dataset]['Expert structural annotation score (mean), random round up/down'],
        df_expert[df_expert.Dataset==dataset]['Combined organizational score (rounded)']
    )

    dims = (4, 4)
    fig, ax = plt.subplots(figsize=dims)
    conf_mat_plot = sns.heatmap(
        conf_mat,
        annot=True,
        fmt="d",
        square=True,
        cmap="Blues",
        vmin=0,
        vmax=1000,
        cbar=False,
    )

    conf_mat_plot.set(
        xlabel='Combined organizational score (rounded)',
        ylabel='Expert score (mean)',
    );
    conf_mat_plot.set_xticklabels([1,2,3,4,5])
    conf_mat_plot.set_yticklabels([1,2,3,4,5]);

    # save png and svg
    if SAVE:
        plt.savefig(save_dir_pngs/f"expert_mean_COS_conf_mat_{dataset}.png", dpi=300, bbox_inches = "tight")
        plt.savefig(save_dir_svgs/f"expert_mean_COS_conf_mat_{dataset}.svg", format="svg", bbox_inches = "tight")

### expert (both) - COS confustion matrix

In [ ]:
from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(
    df_expert_melt['Expert score'],
    df_expert_melt['Combined organizational score (rounded)']
)

dims = (4, 4)
fig, ax = plt.subplots(figsize=dims)
conf_mat_plot = sns.heatmap(
    conf_mat,
    annot=True,
    fmt="d",
    square=True,
    cmap="Blues",
    vmin=0,
    vmax=1000,
    cbar=False,
)

conf_mat_plot.set(
    xlabel='Combined organizational score (rounded)',
    ylabel='Expert score (both)',
);
conf_mat_plot.set_xticklabels([1,2,3,4,5])
conf_mat_plot.set_yticklabels([1,2,3,4,5]);

# save png and svg
if SAVE:
    plt.savefig(save_dir_pngs/'expert_both_COS_conf_mat.png', dpi=300, bbox_inches = "tight")
    plt.savefig(save_dir_svgs/'expert_both_COS_conf_mat.svg', format="svg", bbox_inches = "tight")

In [ ]:
for dataset in df_expert.Dataset.unique():

    conf_mat = confusion_matrix(
        df_expert_melt[df_expert_melt.Dataset==dataset]['Expert score'],
        df_expert_melt[df_expert_melt.Dataset==dataset]['Combined organizational score (rounded)']
    )

    dims = (4, 4)
    fig, ax = plt.subplots(figsize=dims)
    conf_mat_plot = sns.heatmap(
        conf_mat,
        annot=True,
        fmt="d",
        square=True,
        cmap="Blues",
        vmin=0,
        vmax=1000,
        cbar=False,
    )

    conf_mat_plot.set(
        xlabel='Combined organizational score (rounded)',
        ylabel='Expert score (both)',
    );
    conf_mat_plot.set_xticklabels([1,2,3,4,5])
    conf_mat_plot.set_yticklabels([1,2,3,4,5]);

    # save png and svg
    if SAVE:
        plt.savefig(save_dir_pngs/f"expert_both_COS_conf_mat_{dataset}.png", dpi=300, bbox_inches = "tight")
        plt.savefig(save_dir_svgs/f"expert_both_COS_conf_mat_{dataset}.svg", format="svg", bbox_inches = "tight")

## Scatter / bar plots

### cos vs transcript

#### melt fish data for tidy plots

In [ ]:
melt_feats = {
    "(count/μm^2)" : [c for c in df.columns if c.endswith("(count/μm^2)")]
}

melted_dfs = {
    feat:df[df.Type == 'FISH'].melt(
        id_vars=[c for c in df.columns if c not in cols],
        value_vars=cols,
        var_name="FISH probe",
        value_name=f"FISH probe {feat}",
    ).dropna(
        subset=[f"FISH probe {feat}"]
    ).reset_index(
        drop=True
    )
    for feat, cols in melt_feats.items()
}

for feat,v in melted_dfs.items():
    melted_dfs[feat]["FISH probe"] = melted_dfs[feat]["FISH probe"].str.split().str.get(0)

# TODO if counts make it back in will need to merge
df_fish_tidy = melted_dfs["(count/μm^2)"]

In [ ]:
assert 2*len(df[df.Type == 'FISH']) == len(df_fish_tidy)

## Plot FISH vs cos

In [ ]:
DAY_18_COLOR = "#0098EA"
DAY_25_COLOR = "#f5c056"
DAY_32_COLOR = "#FFB2FF"
DAY_COLORS = [DAY_18_COLOR, DAY_32_COLOR]
DAY_COLOR_PALETTE = sns.color_palette(DAY_COLORS)
DAY_COLORS_3 = [DAY_18_COLOR, DAY_25_COLOR, DAY_32_COLOR]
DAY_COLOR_PALETTE_3 = sns.color_palette(DAY_COLORS_3)

### Define bootstrap corr data wrabgling function for ebars on bar plots

In [ ]:
def pack_complex_corr_col(arg_df, arg_col1, arg_col2, outcol="Corr"):
    arg_df[outcol] = arg_df.apply(
        lambda row: np.complex(row[arg_col1], row[arg_col2]), axis="columns"
    )

def spearman_on_complex_col(tuple_col):
    return spearmanr(np.real(tuple_col), np.imag(tuple_col))[0]

### scatter plots

In [ ]:
g = sns.FacetGrid(
    data=df_fish_tidy.sample(frac=1, replace=False).reset_index(drop=True),
    col="FISH probe",
    col_order=PROBE_ORDER+['ACTN2', 'TTN'],
    col_wrap=4,
    hue="Cell age",
    hue_order=[18, 25, 32],
    palette=DAY_COLOR_PALETTE_3,
    sharex=True,
    xlim=(1,5),
    sharey=False,
    height=3.0,
    aspect=1.0,
)

g = g.map(
    plt.scatter,
    "Combined organizational score",
    "FISH probe (count/μm^2)",
    s=10,
    alpha=0.5,
    linewidth=0,
).add_legend()

g.set_titles("{col_name}")

sns.despine()

# save png and svg
if SAVE:
    plt.savefig(save_dir_pngs/'fig_5a_probe_density_vs_org_score.png', dpi=300, bbox_inches = "tight")
    plt.savefig(save_dir_svgs/'fig_5a_probe_density_vs_org_score.svg', format="svg", bbox_inches = "tight")

### bar plots

In [ ]:
pack_complex_corr_col(
    df_fish_tidy,
    "Combined organizational score",
    "FISH probe (count/μm^2)",
    outcol="Corr(COS, probe density)",
)

plt.figure(figsize=(16, 4))
sns.barplot(
    data=df_fish_tidy.dropna(subset=["Combined organizational score", "FISH probe (count/μm^2)"]),
    x="FISH probe",
    y="Corr(COS, probe density)",
    order=PROBE_ORDER+['ACTN2', 'TTN'],
    hue="Cell age",
    hue_order=[18, 25, 32],
    palette=DAY_COLOR_PALETTE_3,
    estimator=spearman_on_complex_col,
    n_boot=1000,
    ci=95,
    errwidth=1,
    capsize=0.1,
);
sns.despine();
plt.legend(title="Cell age", bbox_to_anchor=(1.0, 0.7), frameon=False);

# save png and svg
if SAVE:
    plt.savefig(save_dir_pngs/'fig_5b_probe_density_vs_org_score_by_day.png', dpi=300, bbox_inches = "tight")

In [ ]:
plt.figure(figsize=(16, 4))
sns.barplot(
    data=df_fish_tidy.dropna(subset=["Combined organizational score", "FISH probe (count/μm^2)"]),
    x="FISH probe",
    y="Corr(COS, probe density)",
    order=PROBE_ORDER+['ACTN2', 'TTN'],
    color="grey",
    estimator=spearman_on_complex_col,
    n_boot=1000,
    ci=95,
    errwidth=1,
    capsize=0.1,
);
sns.despine();

# save png and svg
if SAVE:
    plt.savefig(save_dir_pngs/'fig_5b_probe_density_vs_org_score_all_days.png', dpi=300, bbox_inches = "tight")

## All cells protein vs COS

In [ ]:
df["alpha-actinin protein intensity (density)"] = df[
    "alpha-actinin protein intensity (sum)"
]/df["Cell area (μm^2)"]
df["alpha-actinin protein intensity (density, background subtracted)"] = df[
    "alpha-actinin protein intensity (sum, background subtracted)"
]/df["Cell area (μm^2)"]

### Check which protein metric is best overall

In [ ]:
df.groupby(["Dataset", "Cell age",]).corr(
    method="spearman"
).filter(
    [c for c in df.columns if "alpha-actinin" in c]
).rename_axis(
        ["Dataset", "Cell age", "feature"]
).reset_index(
).query(
    'feature == "Combined organizational score"'
).dropna()

### COS vs protein density (all cells)

In [ ]:
g = sns.FacetGrid(
    data=df.dropna(subset=["alpha-actinin protein intensity (sum, background subtracted)"]).sample(frac=1, replace=False).reset_index(drop=True),
    col="Dataset",
    col_order=sorted(df.dropna(subset=["alpha-actinin protein intensity (sum)"]).Dataset.unique()),
    col_wrap=2,
    hue="Cell age",
    hue_order=[18, 25, 32],
    palette=DAY_COLOR_PALETTE_3,
    sharex=True,
    xlim=(1,5),
    sharey=False,
    height=3.0,
    aspect=1.0,
)
g.map(
    sns.scatterplot,
    "Combined organizational score",
    "alpha-actinin protein intensity (density, background subtracted)",
    s=10,
    alpha=0.5,
    linewidth=0,
);

g.set_axis_labels(y_var="Protein density")

# save png and svg
if SAVE:
    plt.savefig(save_dir_pngs/'fish_plus_live_protein_vs_org_score.png', dpi=300, bbox_inches = "tight")

In [ ]:
pack_complex_corr_col(
    df,
    "Combined organizational score",
    "alpha-actinin protein intensity (sum, background subtracted)",
    outcol="Corr(Protein, COS)",
)

plt.figure(figsize=(4,3))
sns.barplot(
    data=df.dropna(subset=["Combined organizational score", "alpha-actinin protein intensity (sum, background subtracted)"]),
    x="Dataset",
    y="Corr(Protein, COS)",
    hue="Cell age",
    hue_order=[18, 25, 32],
    palette=DAY_COLOR_PALETTE_3,
    estimator=spearman_on_complex_col,
    n_boot=1000,
    ci=95,
    errwidth=1,
    capsize=0.1,
);
sns.despine();
plt.legend(title="Cell age", bbox_to_anchor=(1.0, 0.7), frameon=False);

# save png and svg
if SAVE:
    plt.savefig(save_dir_pngs/'fish_plus_live_protein_vs_org_score_corrs.png', dpi=300, bbox_inches = "tight")

## just ACTN2 cells

In [ ]:
df_actn2_reps = df[df.Dataset.isin(['ACTN_OldFish', 'ACTN_NewFish'])].copy().reset_index(drop=True)

### transcript-COS

In [ ]:
g = sns.FacetGrid(
    data=df_actn2_reps,
    col="replate_date",
    col_order=sorted(df_actn2_reps.replate_date.unique()),
    hue="Dataset",
    hue_order=sorted(df_actn2_reps.Dataset.unique()),
    sharex=True,
    sharey=True,
    height=3.0,
    aspect=1.0,
)
g.map(
    sns.scatterplot,
    "Combined organizational score",
    "ACTN2 (count/μm^2)",
    s=10,
    alpha=0.75,
    linewidth=0,
);

g.add_legend()
sns.despine();

# save png and svg
if SAVE:
    plt.savefig(save_dir_pngs/'actn2_only_transcript_vs_org_score_by_replate.png', dpi=300, bbox_inches = "tight")

In [ ]:
pack_complex_corr_col(
    df_actn2_reps,
    "Combined organizational score",
    "ACTN2 (count/μm^2)",
    outcol="Corr(COS, transcript density)",
)

plt.figure(figsize=(4,3))
sns.barplot(
    data=df_actn2_reps,
    x="replate_date",
    order=sorted(df_actn2_reps.replate_date.unique()),
    y="Corr(COS, transcript density)",
    hue="Dataset",
    hue_order=sorted(df_actn2_reps.Dataset.unique()),
    estimator=spearman_on_complex_col,
    n_boot=1000,
    ci=95,
    errwidth=1,
    capsize=0.1,
);
g.add_legend()
sns.despine();
plt.legend(title="Dataset", bbox_to_anchor=(1.0, 0.7), frameon=False);

# save png and svg
if SAVE:
    plt.savefig(save_dir_pngs/'actn2_only_transcript_vs_org_score_by_replate_corr.png', dpi=300, bbox_inches = "tight")

### protein-COS

In [ ]:
g = sns.FacetGrid(
    data=df_actn2_reps,
    col="replate_date",
    col_order=sorted(df_actn2_reps.replate_date.unique()),
    hue="Dataset",
    hue_order=sorted(df_actn2_reps.Dataset.unique()),
    sharex=True,
    sharey=True,
    height=3.0,
    aspect=1.0,
)
g.map(
    sns.scatterplot,
    "Combined organizational score",
    "alpha-actinin protein intensity (density, background subtracted)",
    s=10,
    alpha=0.75,
    linewidth=0,
);

g.set_axis_labels(y_var="Protein density")
g.add_legend()
sns.despine();

# save png and svg
if SAVE:
    plt.savefig(save_dir_pngs/'actn2_only_protein_vs_org_score_by_replate.png', dpi=300, bbox_inches = "tight")

In [ ]:
pack_complex_corr_col(
    df_actn2_reps,
    "Combined organizational score",
    "alpha-actinin protein intensity (density, background subtracted)",
    outcol="Corr(COS, protein density)",
)

plt.figure(figsize=(4,3))
sns.barplot(
    data=df_actn2_reps,
    x="replate_date",
    order=sorted(df_actn2_reps.replate_date.unique()),
    y="Corr(COS, protein density)",
    hue="Dataset",
    hue_order=sorted(df_actn2_reps.Dataset.unique()),
    estimator=spearman_on_complex_col,
    n_boot=1000,
    ci=95,
    errwidth=1,
    capsize=0.1,
);
g.add_legend()
sns.despine();
plt.legend(title="Dataset", bbox_to_anchor=(1.0, 0.7), frameon=False);

# save png and svg
if SAVE:
    plt.savefig(save_dir_pngs/'actn2_only_protein_vs_org_score_by_replate_corr.png', dpi=300, bbox_inches = "tight")

### protein-transcript

In [ ]:
g = sns.FacetGrid(
    data=df_actn2_reps,
    col="replate_date",
    col_order=sorted(df_actn2_reps.replate_date.unique()),
    hue="Dataset",
    hue_order=sorted(df_actn2_reps.Dataset.unique()),
    sharex=True,
    sharey=True,
    height=3.0,
    aspect=1.0,
)
g.map(
    sns.scatterplot,
    "alpha-actinin protein intensity (density, background subtracted)",
    "ACTN2 (count/μm^2)",
    s=10,
    alpha=0.75,
    linewidth=0,
);
g.set_axis_labels(x_var="Protein density")
g.add_legend()
sns.despine();

# save png and svg
if SAVE:
    plt.savefig(save_dir_pngs/'actn2_only_protein_vs_transcript_replate.png', dpi=300, bbox_inches = "tight")

In [ ]:
pack_complex_corr_col(
    df_actn2_reps,
    "ACTN2 (count/μm^2)",
    "alpha-actinin protein intensity (density, background subtracted)",
    outcol="Corr(transcript density, protein density)",
)

plt.figure(figsize=(4,3))
sns.barplot(
    data=df_actn2_reps,
    x="replate_date",
    order=sorted(df_actn2_reps.replate_date.unique()),
    y="Corr(transcript density, protein density)",
    hue="Dataset",
    hue_order=sorted(df_actn2_reps.Dataset.unique()),
    estimator=spearman_on_complex_col,
    n_boot=1000,
    ci=95,
    errwidth=1,
    capsize=0.1,
);
g.add_legend()
sns.despine();
plt.legend(title="Dataset", bbox_to_anchor=(1.0, 0.7), frameon=False);

# save png and svg
if SAVE:
    plt.savefig(save_dir_pngs/'actn2_only_protein_vs_transcript_by_replate_corr.png', dpi=300, bbox_inches = "tight")

## Direct replicates

In [ ]:
df_fish_tidy_reps = data=df_fish_tidy[
    df_fish_tidy.Dataset=="ACTN_NewFish"
]

In [ ]:
g = sns.FacetGrid(
    data=df_fish_tidy_reps.sample(frac=1, replace=False).reset_index(drop=True),
    col="FISH probe",
    col_order=['MYH6', 'MYH7', 'ACTN2', 'TTN'],
    hue="replate_date",
    hue_order=sorted(df_fish_tidy_reps.replate_date.unique()),
    palette="Set2",
    sharex=True,
    xlim=(1,5),
    sharey=False,
    height=3.0,
    aspect=1.0,
)

g = g.map(
    plt.scatter,
    "Combined organizational score",
    "FISH probe (count/μm^2)",
    s=10,
    alpha=0.75,
    linewidth=0,
).add_legend()

g.set_titles("{col_name}")

sns.despine()

# save png and svg
if SAVE:
    plt.savefig(save_dir_pngs/'d25_replicates_probe_density_vs_org_score.png', dpi=300, bbox_inches = "tight")

In [ ]:
sns.catplot(
    kind="violin",
    split=True,
    data=df_fish_tidy_reps,
    x="FISH probe",
    order=['MYH6', 'MYH7', 'ACTN2', 'TTN'],
    y="FISH probe (count/μm^2)",
    hue="replate_date",
    hue_order=sorted(df_fish_tidy_reps.replate_date.unique()),
    palette="Set2",
    height=3,
    aspect=2,
)
sns.despine()

# save png and svg
if SAVE:
    plt.savefig(save_dir_pngs/'d25_replicates_probe_density_violins.png', dpi=300, bbox_inches = "tight")

In [ ]:
sns.catplot(
    kind="violin",
    split=True,
    data=df_fish_tidy_reps,
    x="FISH probe",
    order=['MYH6', 'MYH7', 'ACTN2', 'TTN'],
    y="Combined organizational score",
    hue="replate_date",
    hue_order=sorted(df_fish_tidy_reps.replate_date.unique()),
    palette="Set2",
    height=3,
    aspect=2,
)

sns.despine()

# save png and svg
if SAVE:
    plt.savefig(save_dir_pngs/'d25_replicates_org_score_violins.png', dpi=300, bbox_inches = "tight")

In [ ]:
pack_complex_corr_col(
    df_fish_tidy_reps,
    "Combined organizational score",
    "FISH probe (count/μm^2)",
    outcol="Corr(Cos, transcript density)",
)

plt.figure(figsize=(4,3))
sns.barplot(
    data=df_fish_tidy_reps,
    x="FISH probe",
    order=['MYH6', 'MYH7', 'ACTN2', 'TTN'],
    y="Corr(Cos, transcript density)",
    hue="replate_date",
    hue_order=sorted(df_fish_tidy_reps.replate_date.unique()),
    palette="Set2",
    estimator=spearman_on_complex_col,
    n_boot=1000,
    ci=95,
    errwidth=1,
    capsize=0.1,
);
sns.despine();
plt.legend(title="replate_date", bbox_to_anchor=(1.0, 0.7), frameon=False);


# # save png and svg
# if SAVE:
#     plt.savefig(save_dir_pngs/'d25_replicates_probe_density_vs_org_score_corrs.png', dpi=300, bbox_inches = "tight")

## EDA distribution / ANOVA stuff

### Live vs FISH in aggregate

In [ ]:
g = sns.catplot(
    data=df.melt(
        id_vars=[c for c in df.columns if c not in feat_cols+["Combined organizational score"]],
        value_vars=feat_cols+["Combined organizational score"],
        var_name="Feature"
    ),
    height=2.5,
    aspect=1.25,
    sharey=False,
    kind="violin",
    linewidth=0.5,
    split=True,
    col="Feature",
    col_wrap=4,
    x="Cell age",
    y='value',
    hue='Type',
    palette="Set2",
)

# save png and svg
if SAVE:
    plt.savefig(save_dir_pngs/'all_features_violins_by_experiment_type.png', dpi=300, bbox_inches = "tight")
    plt.savefig(save_dir_svgs/'all_features_violins_by_experiment_type.svg', format="svg", bbox_inches = "tight")

### COS by dataset and cell age

In [ ]:
feature='Combined organizational score'

g = sns.catplot(
    data=df.rename(columns={"ge_wellID":"Well ID"}),
    height=3,
    aspect=1,
    dodge=False,
    sharex=False,
    sharey=False,
    kind="violin",
    inner=None,
    linewidth=0.5,
    col="Cell age",
    x="Dataset",
    y=feature,
    hue="Dataset",
    hue_order=['OldFish', 'NewFish', 'Transcript-Protein', 'Live'],
)

for i, age in enumerate(sorted(df["Cell age"].unique())):
    sns.pointplot(
        ax=g.axes[0,i],
        data=df[df["Cell age"]==age].rename(columns={"ge_wellID":"Well ID"}),
        x="Dataset",
        y=feature,
        color='black',
        scale=0.5,
        estimator=np.median,
        ci=None,
        join=False,
    )


plt.subplots_adjust(top=0.8)
g.fig.suptitle('COS distribution across Datasets', x="0.5");

# save png and svg
if SAVE:
    plt.savefig(save_dir_pngs/'COS_violins_by_dataset.png', dpi=300, bbox_inches = "tight")
    plt.savefig(save_dir_svgs/'COS_violins_by_dataset.svg', format="svg", bbox_inches = "tight")

In [ ]:
feature='Combined organizational score'

g = sns.catplot(
    data=df.rename(columns={"ge_wellID":"Well ID"}),
    height=3,
    aspect=1,
    dodge=False,
    sharex=False,
    sharey=False,
    kind="bar",
    ci="sd",
    errwidth=1,
    capsize=0.5,
    linewidth=1,
    col="Cell age",
    x="Dataset",
    y=feature,
    hue="Dataset",
    hue_order=['OldFish', 'NewFish', 'Transcript-Protein', 'Live'],
)

plt.subplots_adjust(top=0.8)
g.fig.suptitle('COS distribution across Datasets', x="0.5");

# save png and svg
if SAVE:
    plt.savefig(save_dir_pngs/'COS_boxes_by_dataset.png', dpi=300, bbox_inches = "tight")
    plt.savefig(save_dir_svgs/'COS_boxes_by_dataset.svg', format="svg", bbox_inches = "tight")

In [ ]:
df.groupby(
    ["Cell age", 'Dataset']
)[
    'Combined organizational score'
].agg(
    ['median', 'mean', 'std']
).round(
    2
).reset_index()

In [ ]:
df[["Dataset","ge_wellID"]].drop_duplicates().merge(
    df[
        df["Cell age"]==18
    ].groupby(
        ['ge_wellID']
    )[
        'Combined organizational score'
    ].agg(
        ['median', 'mean', 'std']
    ).round(
        2
    ).reset_index(
    )
).sort_values(
    "mean",
    ascending=False
)

### COS across wells

In [ ]:
feature='Combined organizational score'

g = sns.catplot(
    data=df.rename(columns={"ge_wellID":"Well ID"}),
    height=2.5,
    aspect=5,
    dodge=False,
    sharex=False,
    sharey=False,
    kind="violin",
    inner=None,
    linewidth=0.5,
    row="Cell age",
    x="Well ID",
    y=feature,
    hue="Dataset",
    hue_order=['OldFish', 'NewFish', 'Transcript-Protein', 'Live'],
)

for i, age in enumerate(sorted(df["Cell age"].unique())):
    sns.pointplot(
        ax=g.axes[i,0],
        data=df[df["Cell age"]==age].rename(columns={"ge_wellID":"Well ID"}),
        x="Well ID",
        y=feature,
        color='black',
        scale=0.5,
        estimator=np.median,
        ci=None,
        join=False,
    )

g.set(xticklabels=[])

plt.subplots_adjust(top=0.9)
g.fig.suptitle('COS distribution across wells', x="0.4");

# save png and svg
if SAVE:
    plt.savefig(save_dir_pngs/'COS_violins_by_well.png', dpi=300, bbox_inches = "tight")
    plt.savefig(save_dir_svgs/'COS_violins_by_well.svg', format="svg", bbox_inches = "tight")

### COS across wells (colored by plate)

In [ ]:
feature='Combined organizational score'

g = sns.catplot(
    data=df.rename(columns={"ge_wellID":"Well ID"}),
    height=2.5,
    aspect=5,
    dodge=False,
    sharex=False,
    sharey=False,
    kind="violin",
    inner=None,
    linewidth=0.5,
    row="Cell age",
    x="Well ID",
    y=feature,
    hue="plate_name",
)

for i, age in enumerate(sorted(df["Cell age"].unique())):
    sns.pointplot(
        ax=g.axes[i,0],
        data=df[df["Cell age"]==age].rename(columns={"ge_wellID":"Well ID"}),
        x="Well ID",
        y=feature,
        color='black',
        scale=0.5,
        estimator=np.median,
        ci=None,
        join=False,
    )

g.set(xticklabels=[])

plt.subplots_adjust(top=0.9)
g.fig.suptitle('COS distribution across wells', x="0.45");

# save png and svg
if SAVE:
    plt.savefig(save_dir_pngs/'COS_violins_by_well_plate_color.png', dpi=300, bbox_inches = "tight")
    plt.savefig(save_dir_svgs/'COS_violins_by_well_plate_color.svg', format="svg", bbox_inches = "tight")

### Cell area across wells

## ANOVA

### first look at cell age

In [ ]:
pd.concat(
    [
        pg.anova(
            data=df.rename(
                columns={
                    "Cell age":"Cell_age",
                    'Combined organizational score':'Combined_organizational_score',
                }
            ),
            dv='Combined_organizational_score',
            between=between
        ) for between in ["Type","Dataset","Cell_age","ge_wellID"]
    ]
).reset_index(drop=True)

### Now look within each age at each other variable

In [ ]:
dfs_anova = []
for between in ["Type","Dataset","ge_wellID"]:
    for age in sorted(df["Cell age"].unique()):
        if len(df[df["Cell age"]==age][between].drop_duplicates()) > 1:
            df_anova = pg.anova(
                data=df[df["Cell age"]==age].rename(
                    columns={
                        "Cell age":"Cell_age",
                        'Combined organizational score':'Combined_organizational_score',
                    }
                ),
                dv='Combined_organizational_score',
                between=between.replace(" ", "_"),
                detailed=True
            )
            df_anova["Cell age"] = age
            dfs_anova += [df_anova]
    
df_anova_within_age = pd.concat(dfs_anova).reset_index(drop=True)
df_anova_within_age[df_anova_within_age.Source != "Within"]

### Largest variance explained by factors is all in day 18, so look closer there

#### variance explained by well when looking at all datasets in day 18

In [ ]:
dfs_day_18_anova = []
for dataset in df[df["Cell age"]==18].Dataset.unique():
    df_day_18_anova = pg.anova(
        data=df[(df["Cell age"]==18) & (df["Dataset"]==dataset)].rename(
            columns={
                "Cell age":"Cell_age",
                'Combined organizational score':'Combined_organizational_score',
            }
        ),
        dv='Combined_organizational_score',
        between=[
            "ge_wellID",
        ],
        detailed=True
    )
    df_day_18_anova["Dataset"] = dataset
    dfs_day_18_anova += [df_day_18_anova]
    
df_within_day_18_anova = pd.concat(dfs_day_18_anova)
df_within_day_18_anova[df_within_day_18_anova.Source!="Within"].reset_index(drop=True)

In [ ]:
dfs_homoscedasticity = []
for group in ["Type","Dataset","Cell age","ge_wellID"]:
    df_homoscedasticity = pg.homoscedasticity(df, dv='Combined organizational score', group=group, method='levene')
    df_homoscedasticity["group"] = group
    dfs_homoscedasticity += [df_homoscedasticity] 
df_homoscedasticity = pd.concat(dfs_homoscedasticity).reset_index(drop=True)
df_homoscedasticity

## split violin plots of paired fixed / live by replate_date

In [ ]:
plate_matches_fish_live = {
    5500000013:[5500000094, 5500000096],
    5500000014:[5500000099],
    5500000172:[5500000111, 5500000113],
    5500000171:[5500000114]
}

df["matched_plates"] = np.nan
for i, (fish_plate, live_plates) in enumerate(plate_matches_fish_live.items()):
    mdata = df[df.plate_name==fish_plate][["Cell age", "replate_date"]].drop_duplicates().squeeze()
    assert mdata.shape == (2,)
    df.loc[df.plate_name==fish_plate, "matched_plates"] = f"{mdata['replate_date']}_day_{mdata['Cell age']}"
    for live_plate in live_plates:
        df.loc[df.plate_name==live_plate, "matched_plates"] = f"{mdata['replate_date']}_day_{mdata['Cell age']}"

In [ ]:
g = sns.catplot(
    data=df.dropna(subset=["matched_plates"]).rename(columns={"replate_date":"Replate date"}),
    height=2.5,
    aspect=1.25,
    sharey=False,
    kind="violin",
    linewidth=0.5,
    split=True,
    col="Replate date",
    x="Cell age",
    y="Combined organizational score",
    hue='Type',
    palette="Set2",
)
plt.subplots_adjust(top=0.8)
g.fig.suptitle('Matched FISH/Live plates', x="0.5");

# save png and svg
if SAVE:
    plt.savefig(save_dir_pngs/'matched_fish_live_plates.png', dpi=300, bbox_inches = "tight")
    plt.savefig(save_dir_svgs/'matched_fish_live_plates.svg', format="svg", bbox_inches = "tight")